### Data Source : CSV

In [1]:
import pandas as pd
df = pd.read_excel(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\sample1.xlsx')

df.to_csv('sample1.csv')

In [2]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os
from langchain.embeddings.openai import OpenAIEmbeddings

In [3]:
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
# Load the documents
loader = CSVLoader(file_path=r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\1 to Many.csv')

In [6]:
# Create an index using the loaded documents
index_creator = VectorstoreIndexCreator(embedding=embeddings)
docsearch = index_creator.from_loaders([loader])

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain\indexes\vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [7]:
# Create a question-answering chain using the index
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=openai.api_key)
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [8]:
# Pass a query to the chain
query = "Summarize the doument"
response = chain({"question": query})

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
print(type(response['result']))

<class 'str'>


### Document Source: PDF doc

In [2]:
import os
import PyPDF2
from PyPDF2 import PdfReader

# Importing necessary modules from langchain and langchain_openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
pdfpath = r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\Vedanta Cairn_May 2024_PO No.7300175294 (1).PDF'

In [5]:
def chunk_processing(pdf):
    """
    Process a PDF file, extracting text and splitting it into chunks.
    """
    pdf_reader = PdfReader(pdf)
        
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
        
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text=text)
    return chunks

In [6]:
def embeddings(chunks):
    """
    Create embeddings for text chunks using OpenAI.
    """
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
    # Create vector store using FAISS
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

In [9]:
def generation(VectorStore):
    """
    Generate responses based on prompts and embeddings.
    """
    retriever = VectorStore.as_retriever()
    
    # Define template for prompts
    template = """Respond to the prompt based on the following context: {context}
    Questions: {questions}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Initialize ChatOpenAI model
    model = ChatOpenAI(openai_api_key=openai.api_key)
    
    # Define processing chain
    chain = (
        {"context": retriever, "questions": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    
    # Prompt user for input
    query = input("Insert Prompt: ")
    
    # Invoke the processing chain with user input
    output = chain.invoke(query)
    return output

In [11]:
type(generation(embeddings(chunk_processing(pdfpath))))

str

### Document Source: Docx

In [21]:
#Convert the docx to pdf
from docx2pdf import convert

def convert_docx_to_pdf(input_path, output_path):
    try:
        # Convert DOCX to PDF
        convert(input_path, output_path)
        print(f"Conversion complete. PDF saved at {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
convert_docx_to_pdf(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\document_0.docx', 'document_0.pdf')

100%|██████████| 1/1 [00:03<00:00,  3.68s/it]

Conversion complete. PDF saved at document_0.pdf


In [13]:
df =pd.read_csv(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\1 to Many.csv')
df.to_excel('1 to Many.xlsx')

### Creating a standarad Pipeline

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import PyPDF2
from PyPDF2 import PdfReader

# Importing necessary modules from langchain and langchain_openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
#Convert the docx to pdf
from docx2pdf import convert

import pandas as pd

from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

: 

In [9]:
class DataTalk:

    def __init__(self, filepath):
        self.filepath = filepath

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else: 
            file = self.filepath

        return file
    
    
    def generate_response(self, query):
        # Initialize the OpenAI embeddings
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name="gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            # Load the documents
            loader = CSVLoader(file_path=file)
            # Create an index using the loaded documents
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            # Create a question-answering chain using the index
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']
        
        elif file.lower().endswith('.pdf'):

            """
            Process a PDF file, extracting text and splitting it into chunks.
            """
            pdf_reader = PdfReader(file)
                
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
                
            # Split text into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_text(text=text)

            # Create vector store using FAISS
            vector_store = FAISS.from_texts(chunks, embedding=embeddings)

            """
            Generate responses based on prompts and embeddings.
            """
            retriever = vector_store.as_retriever()
            
            # Define template for prompts
            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)

            # Initialize ChatOpenAI model
            model = ChatOpenAI(openai_api_key=openai.api_key)
            
            # Define processing chain
            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )
            
            # Invoke the processing chain with user input
            response = chain.invoke(query)
        
        else:
            print('The Data source format is not in required format')

        return response

            

In [10]:
chat = DataTalk(r'C:\Users\Samruddhi More\Desktop\LLM BE Template\data\document_0.docx')
chat.generate_response('What is the document about?')

c:\Users\Samruddhi More\Desktop\LLM BE Template\tempenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
100%|██████████| 1/1 [00:04<00:00,  4.31s/it]


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [6]:
import os
import sys
import requests
from io import BytesIO
import fitz  # PyMuPDF
from langchain.document_loaders import CSVLoader, PyMuPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from docx2pdf import convert
import pandas as pd
from dotenv import load_dotenv
import openai
from PyPDF2 import PdfReader

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [7]:
class DataTalk:

    def __init__(self, cdn_url):
        self.cdn_url = cdn_url
        self.filepath = self.download_file(cdn_url)

    def download_file(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            cdn_filename = url.split('/')[-1]
            with open(cdn_filename, 'wb') as f:
                f.write(response.content)
            print(f"File {cdn_filename} downloaded successfully.")
            return cdn_filename
        else:
            raise Exception(f"Failed to download file from {url}")

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file, index=False)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else:
            file = self.filepath

        return file

    def generate_response(self, query):
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name = "gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            loader = CSVLoader(file_path=file)
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']

        elif file.lower().endswith('.pdf'):
            pdf_reader = PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
            chunks = text_splitter.split_text(text=text)

            vector_store = FAISS.from_texts(chunks, embedding=embeddings)
            retriever = vector_store.as_retriever()

            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)
            model = ChatOpenAI(openai_api_key=openai.api_key)

            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )

            response = chain.invoke(query)

        else:
            print('The data source format is not in the required format.')

        return response


In [8]:
chat = DataTalk('https://github.com/py-pdf/sample-files/blob/main/001-trivial/minimal-document.pdf')
chat.generate_response('What is the doc about?')

File minimal-document.pdf downloaded successfully.


PdfReadError: EOF marker not found

In [2]:
import os
import requests
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import PyPDF2
from PyPDF2 import PdfReader
from docx2pdf import convert
import pandas as pd
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

c:\Users\Samruddhi More\Desktop\LLM BE Template\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class DataTalk:

    def __init__(self, url):
        self.url = url
        self.filepath = self.download_file(url)

    def download_file(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            # Extract the filename from the URL and save it in the current directory
            filename = url.split("/")[-1].split("?")[0]  # Remove query parameters if any
            with open(filename, "wb") as file:
                file.write(response.content)
            return filename
        else:
            raise Exception(f"Failed to download file from {url}")

    def source_type(self):
        file = ''
        folderpath = os.path.dirname(self.filepath)
        filename = os.path.basename(self.filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(self.filepath)
            df.to_csv(file)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(self.filepath, file)
        else: 
            file = self.filepath

        return file

    def generate_response(self, query):
        # Initialize the OpenAI embeddings
        embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
        model_name = "gpt-3.5-turbo-instruct"
        file = self.source_type()
        response = ''

        if file.endswith('.csv'):
            # Load the documents
            loader = CSVLoader(file_path=file)
            # Create an index using the loaded documents
            index_creator = VectorstoreIndexCreator(embedding=embeddings)
            docsearch = index_creator.from_loaders([loader])
            # Create a question-answering chain using the index
            llm = OpenAI(model_name=model_name, openai_api_key=openai.api_key)
            chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
            response = chain({"question": query})
            response = response['result']

        elif file.lower().endswith('.pdf'):
            """
            Process a PDF file, extracting text and splitting it into chunks.
            """
            pdf_reader = PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            
            # Split text into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_text(text=text)

            # Create vector store using FAISS
            vector_store = FAISS.from_texts(chunks, embedding=embeddings)

            """
            Generate responses based on prompts and embeddings.
            """
            retriever = vector_store.as_retriever()
            
            # Define template for prompts
            template = """Respond to the prompt based on the following context: {context}
            Questions: {questions}
            """
            prompt = ChatPromptTemplate.from_template(template)

            # Initialize ChatOpenAI model
            model = ChatOpenAI(openai_api_key=openai.api_key)
            
            # Define processing chain
            chain = (
                {"context": retriever, "questions": RunnablePassthrough()}
                | prompt
                | model
                | StrOutputParser()
            )
            
            # Invoke the processing chain with user input
            response = chain.invoke(query)
        
        else:
            print('The data source format is not in required format')

        return response

# Example usage
chat = DataTalk('https://github.com/anupsawant/PRDashboard/raw/main/dashboard/data/D0_Monthly.xlsx')
response = chat.generate_response('What is the doc about?')
print(response)

Exception: Failed to download file from https://github.com/anupsawant/PRDashboard/raw/main/dashboard/data/D0_Monthly.xlsx

### Testing loaders

In [64]:
!pip install sentence_transformers

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Samruddhi More\\Desktop\\LLM BE Template\\tempenv\\Lib\\site-packages\\~okenizers\\tokenizers.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for transformers<5.0.0,>=4.34.0 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/5a/6a/775b93d6888c31f1f1fc457e4f5cc89f0984412d5dcdef792b8f2aa6e812/torch-2.4.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.4.1-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/5d/55/0403bf2031250ac98

In [12]:
!pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/3c/60/eccdd92dd4af3e4bea6d6a342f7588c618a15b9bec4b968af581e498bcc4/pypdf-4.3.1-py3-none-any.whl.metadata
  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
!pip install unstructured

  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/58/7b/93126eed91753d65d0c07e9f4c80bd715b6b6003f139483024ae00749aa2/unstructured-0.15.9-py3-none-any.whl.metadata
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Obtaining dependency information for filetype from https://files.pythonhosted.org/packages/18/79/1b8fa1bb3568781e84c9200f951c735f3f157429f44be0495da55894d620/filetype-1.2.0-py2.py3-none-any.whl.metadata
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Obtaining dependency information for python-magic from https://files.pythonhosted.org/packages/6c/73/9f872cb81fc5c3bb48f7227872c28975f998f3e7c2b1c16e95e6432bbb90/python_magic-0.4.27-py2.py3-none-any.whl.metadata
  Using cached python_magic-0.4.27-py2.


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
!pip install langchain-chroma

  Obtaining dependency information for langchain-chroma from https://files.pythonhosted.org/packages/65/22/fa0a518618d769dacc732cf3a623debb04720949e1a5f26d02756c19449f/langchain_chroma-0.1.3-py3-none-any.whl.metadata
  Using cached langchain_chroma-0.1.3-py3-none-any.whl.metadata (1.5 kB)
  Obtaining dependency information for chromadb!=0.5.4,!=0.5.5,<0.6.0,>=0.4.0 from https://files.pythonhosted.org/packages/3e/c9/d55602e07bb839c578f4698b92db7ea88c1aa92f69d4c86076c1597e451b/chromadb-0.5.3-py3-none-any.whl.metadata
  Using cached chromadb-0.5.3-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for fastapi<1,>=0.95.2 from https://files.pythonhosted.org/packages/d0/80/17037f322c280efbc623e341358d38d0299c6ee899d619a879b3593aa6da/fastapi-0.114.0-py3-none-any.whl.metadata
  Using cached fastapi-0.114.0-py3-none-any.whl.metadata (27 kB)
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/91/fd/e4bda6228637ecae5732162b5ac2a5a822


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
# from langchain_groq import ChatGroq
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# from langchain.llms import OpenAI
# from dotenv import load_dotenv
# import openai
import os
# import pandas as pd
from docx2pdf import convert

In [26]:
def source_type( filepath):
        # file = ''
        folderpath = os.path.dirname(filepath)
        filename = os.path.basename(filepath)
        if filename.endswith('.xlsx'):
            csvname = filename.split('.')[0] + '.csv'
            file = os.path.join(folderpath, csvname)
            df = pd.read_excel(filepath)
            df.to_csv(file)
        elif filename.endswith('.docx'):
            pdfname = filename.split('.')[0] + '.pdf'
            file = os.path.join(folderpath, pdfname)
            convert(filepath, file)
        else: 
            file = filepath

        return file

In [43]:
def data_loader( filepath):
        if filepath.endswith('.csv') or filepath.endswith('.docx'):
            file = source_type(filepath)
        else:
            file = filepath
        loader = None
        if file.endswith('.csv'):
            loader = CSVLoader(file_path=file)
        if file.endswith('.pdf'):
            loader = PyPDFLoader(file)
        if 'http' in filepath:
            loader = UnstructuredURLLoader(urls=[file])
        if loader is not None:
            data = loader.load()
            return data
        else:
            raise ValueError("Unsupported file type or URL")

In [71]:
data = data_loader("https://python.langchain.com/v0.2/docs/integrations/document_loaders/url/")

In [57]:
path = r"C:\Users\Samruddhi More\Desktop\LLM BE Template\datatalk_template\jupyter_notebook\minimal-document.pdf"

In [60]:
type(data)

list

In [66]:
!pip install sentence_transformers

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for transformers<5.0.0,>=4.34.0 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [72]:
text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=10,
                length_function=len
            )


chunks = text_splitter.split_documents(data)
embeddings = HuggingFaceBgeEmbeddings()
vector_store = Chroma.from_documents(
    chunks,
    # collection_name="example_collection",
    embeddings,
    persist_directory="./chromadb",  # Where to save data locally, remove if not neccesary
)
retriever = vector_store.as_retriever()

In [68]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [73]:
retriever.get_relevant_documents('PlaywriteURL')

[Document(page_content='loader = PlaywrightURLLoader(urls=urls, remove_selectors=["header", "footer"])\n\ndata = await loader.aload()\n\ndata[0]\n\nAPI Reference:PlaywrightURLLoader', metadata={'source': 'https://python.langchain.com/v0.2/docs/integrations/document_loaders/url/'}),
 Document(page_content='Playwright URL Loader\u200b\n\nPlaywright is an open-source automation tool developed by Microsoft that allows you to programmatically control and automate web browsers. It is designed for end-to-end testing, scraping, and automating tasks across various web browsers such as Chromium, Firefox, and WebKit.\n\nThis covers how to load HTML documents from a list of URLs using the PlaywrightURLLoader.\n\nPlaywright enables reliable end-to-end testing for modern web apps.\n\nAs in the Selenium case, Playwright allows us to load and render the JavaScript pages.\n\nTo use the PlaywrightURLLoader, you have to install playwright and unstructured. Additionally, you have to install the Playwright